## GraphRDF

In [1]:

from langchain_community.graphs import RdfGraph
# from langchain.graphs import RdfGraph
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphSparqlQAChain
graph = RdfGraph(
    query_endpoint="https://enpkg.commons-lab.org/graphdb/repositories/ENPKG",
    standard="rdf"
)


ValueError: You did something wrong formulating either the URI or your SPARQL query

In [30]:
from rdflib import Graph
g = Graph()
g.parse('https://enpkg.commons-lab.org/')

for s, p, o in g:
    print(s, p, o)


PluginException: No plugin registered for (text/html, <class 'rdflib.parser.Parser'>)

In [2]:
graph = RdfGraph(
    query_endpoint="https://enpkg.commons-lab.org/graphdb/repositories/ENPKG",
    standard="rdf",
    local_copy="enpkg.ttl"
)

ValueError: You did something wrong formulating either the URI or your SPARQL query

In [ ]:

graph.get_schema

'In the following, each IRI is followed by the local name and optionally its description in parentheses. \nThe RDF graph supports the following node types:\n<http://www.w3.org/2002/07/owl#Ontology> (Ontology, None), <http://www.w3.org/2000/01/rdf-schema#Class> (Class, None), <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property> (Property, None)\nThe RDF graph supports the following relationships:\n<http://www.w3.org/2000/01/rdf-schema#label> (label, None), <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> (type, None), <http://www.w3.org/2000/01/rdf-schema#comment> (comment, None), <http://www.w3.org/2000/01/rdf-schema#subClassOf> (subClassOf, None), <http://www.w3.org/2000/01/rdf-schema#range> (range, None), <http://purl.org/dc/terms/contributor> (contributor, None), <http://purl.org/dc/terms/title> (title, None), <http://purl.org/dc/terms/license> (license, None), <http://www.w3.org/2002/07/owl#versionInfo> (versionInfo, None), <http://purl.org/dc/terms/description> (description, Non

In [17]:

temperature = 0.3
# model_id = "gpt-4" # don't work with gpt-4 : the request is broken : 
# ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '`'  (at char 0), (line:1, col:1



# https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
model = ChatOpenAI(temperature=temperature, 
                #    model=model_id, # default is 'gpt-3.5-turbo'
                   max_retries=3,
                   )
#https://api.python.langchain.com/en/latest/_modules/langchain/chains/graph_qa/sparql.html#
chain = GraphSparqlQAChain.from_llm(
    model, graph=graph, verbose=True
)

In [9]:
type(chain)
## see https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html?highlight=llmchain#

langchain.chains.graph_qa.sparql.GraphSparqlQAChain

In [10]:
chain.get_graph

<bound method Runnable.get_graph of GraphSparqlQAChain(verbose=True, graph=<langchain_community.graphs.rdf_graph.RdfGraph object at 0x1063bb010>, sparql_generation_select_chain=LLMChain(prompt=PromptTemplate(input_variables=['prompt', 'schema'], template='Task: Generate a SPARQL SELECT statement for querying a graph database.\nFor instance, to find all email addresses of John Doe, the following query in backticks would be suitable:\n```\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nSELECT ?email\nWHERE {{\n    ?person foaf:name "John Doe" .\n    ?person foaf:mbox ?email .\n}}\n```\nInstructions:\nUse only the node types and properties provided in the schema.\nDo not use any node types and properties that are not explicitly provided.\nInclude all necessary prefixes.\nSchema:\n{schema}\nNote: Be as concise as possible.\nDo not include any explanations or apologies in your responses.\nDo not respond to any questions that ask for anything else than for you to construct a SPARQL query.\nDo no

In [18]:
chain.run("""Context: you are a bot interfaced with a rdf graph enpkg. User can ask you in natural langage a question relating to this graph. You transform to SPARQL query and return the result.
          If you get an error message from sparql query, try another one and taking into account the error message. 
          Repeat maximum 3 times the queries. 
          Generated SPARQL query should not contain ` character.
          User: 'How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey of the annotations?'
          """)







> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX enpkg: <http://example.org/enpkg#>
SELECT (COUNT(?feature) AS ?count)
WHERE {
  ?feature rdf:type enpkg:Feature ;
           enpkg:hasAnnotation ?annotation1, ?annotation2 .
  ?annotation1 enpkg:hasInCHIKey ?inchikey .
  ?annotation2 enpkg:hasInCHIKey ?inchikey .
  ?annotation1 enpkg:hasAnnotationType "SIRIUS/CSI:FingerID" .
  ?annotation2 enpkg:hasAnnotationType "ISDB" .
  ?feature enpkg:hasIonizationMode "pos ionization" ;
           enpkg:hasIonizationMode "neg ionization" .
}
Full Context:
[(rdflib.term.Literal('0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)]

> Finished chain.


'Based on the information provided, the SPARQL query has returned a result of 0. This means that there are no features with the same SIRIUS/CSI:FingerID and ISDB annotation when comparing the InCHIKey of the annotations.'

### get classes from enpkg

In [1]:
# general python libs
import json
import os
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE
import glob
import rdflib
from rdflib import Graph
import tiktoken
import time

In [2]:
endpoint_url = 'https://enpkg.commons-lab.org/graphdb/repositories/ENPKG'


def run_sparql(query, 
               url=endpoint_url):
    sparql = SPARQLWrapper(url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(600)
    results = sparql.query().convert()
    results = results['results']['bindings']
    return results

### extracting Classes

#### 1st query

In [13]:
query = """  
select distinct ?o where { 
	?s a ?o .
} 
"""

In [14]:
run_sparql(query)

[{'o': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'}},
 {'o': {'type': 'uri',
   'value': 'http://www.w3.org/2002/07/owl#TransitiveProperty'}},
 {'o': {'type': 'uri',
   'value': 'http://www.w3.org/2002/07/owl#SymmetricProperty'}},
 {'o': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#List'}},
 {'o': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#Class'}},
 {'o': {'type': 'uri',
   'value': 'http://www.w3.org/2000/01/rdf-schema#Datatype'}},
 {'o': {'type': 'uri',
   'value': 'http://www.w3.org/2000/01/rdf-schema#ContainerMembershipProperty'}},
 {'o': {'type': 'uri',
   'value': 'https://enpkg.commons-lab.org/kg/RawMaterial'}},
 {'o': {'type': 'uri', 'value': 'https://enpkg.commons-lab.org/kg/LabObject'}},
 {'o': {'type': 'uri', 'value': 'http://rdfs.org/ns/void#Dataset'}},
 {'o': {'type': 'uri',
   'value': 'https://enpkg.commons-lab.org/kg/LabExtract'}},
 {'o': {'type': 'uri', 'value': 'https://enpkg.commons

#### 2nd query

In [3]:
prefixes = {
    "owl": """PREFIX owl: <http://www.w3.org/2002/07/owl#>\n""",
    "rdf": """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n""",
    "rdfs": """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n""",
    "xsd": """PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n""",
}

query = prefixes["rdfs"] + (
    """SELECT DISTINCT ?cls ?com\n"""
    """WHERE { \n"""
    """    ?instance a ?cls . \n"""
    """    OPTIONAL { ?cls rdfs:comment ?com } \n"""
    """}"""
)


In [4]:
res = run_sparql(query)
print(res)

[{'cls': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'}}, {'cls': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#TransitiveProperty'}}, {'cls': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#SymmetricProperty'}}, {'cls': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#List'}}, {'cls': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#Class'}}, {'cls': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#Datatype'}}, {'cls': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#ContainerMembershipProperty'}}, {'cls': {'type': 'uri', 'value': 'https://enpkg.commons-lab.org/kg/RawMaterial'}, 'com': {'type': 'literal', 'value': 'A raw laboratory biological material, i.e. before extraction'}}, {'cls': {'type': 'uri', 'value': 'https://enpkg.commons-lab.org/kg/LabObject'}, 'com': {'type': 'literal', 'value': 'An object that correspond to a physical laboratory object'}}, {'cls': {'type': 'u

### extracting relationships

In [4]:
rel_query_rdf = prefixes["rdfs"] + (
    """SELECT DISTINCT ?rel ?com\n"""
    """WHERE { \n"""
    """    ?subj ?rel ?obj . \n"""
    """    OPTIONAL { ?rel rdfs:comment ?com } \n"""
    """}"""
)


In [5]:
run_sparql(query)

HTTPError: HTTP Error 502: Proxy Error

In [ ]:
from rdflib.query import ResultRow

## Sparql chain

In [ ]:
# refaire cette classe https://api.python.langchain.com/en/latest/_modules/langchain/chains/graph_qa/sparql.html#

## Agent type?

In [ ]:
"""
ZERO_SHOT_REACT_DESCRIPTION: This is a zero-shot agent that performs a reasoning step before acting. It's suitable for scenarios where an immediate response is required without prior training.
OPENAI_FUNCTIONS: This is an agent optimized for using open AI functions. It's suitable for scenarios where open AI functions are required.
"""


#  https://python.langchain.com/docs/modules/agents/
# "this example, we will use OpenAI Function Calling to create this agent. 
# This is generally the most reliable way to create agents."



## Vector stores + Agent

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

## Message Placeholder

In [4]:
from langchain.prompts import  MessagesPlaceholder

MessagesPlaceholder().get_prompt()


ValidationError: 1 validation error for MessagesPlaceholder
variable_name
  field required (type=value_error.missing)

## Caching
https://python.langchain.com/docs/modules/model_io/models/chat/chat_model_caching

In [7]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

ModuleNotFoundError: No module named 'langchain.globals'

## Output parser

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

## Why agents?
The core idea of agents is to use an LLM to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
https://python.langchain.com/docs/modules/agents/